# Comparison of Sentinel-1 RTC products from different software

This notebook was produced as part of the Digital Earth Antarctica (DEAnt) evaluation of different software options to produce SAR RTC data. Four options have been compares using 'on-the-fly' (otf) pipelines developed for each software. The four softwares options of interest are:

In [ ]:
import os
import h5py
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import boto3
import pandas as pd
import json
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
import rioxarray
import asf_search as asf
from shapely.geometry import Polygon
from celluloid import Camera # getting the camera
from IPython.display import HTML
from itertools import combinations

from utils.compare import (make_difference_gif, 
                           assign_crs, 
                           plot_tifs, 
                           plot_difference_maps, 
                           reproject_match_tifs, 
                           plot_histograms, 
                           plot_array)

sns.set_theme()
%matplotlib inline

# Settings

In [ ]:
# general structure for scenes in s3
# s3_bucket/software/dem/scene/scene_files
s3_bucket = 'deant-data-public-dev'
s3_bucket_link = 'https://deant-data-public-dev.s3.ap-southeast-2.amazonaws.com/'

scenes = [
        # Antarctica - Maitri Station (-70.76683, 11.7308)
        'S1B_IW_SLC__1SSH_20190327T195016_20190327T195045_015544_01D236_9504',
        'S1B_IW_SLC__1SSH_20190526T195018_20190526T195048_016419_01EE8D_53BC',
        'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B',
        # Antarctica - Bharati Station (-69.4068, 76.19525)
        'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63',
        'S1A_IW_SLC__1SSH_20190605T222724_20190605T222751_027550_031BE1_AD3A',
        # Antarctica - lutzow-holm bay (-69.6901, 39.4502)
        'S1A_IW_SLC__1SSH_20220223T175626_20220223T175653_042043_05021A_BB8E',
        'S1A_IW_SLC__1SSH_20221021T175636_20221021T175703_045543_0571D7_989F',
        # Antarctica - Erebus Ice Tongue (-77.7018, 166.7541) 
        'S1A_IW_SLC__1SSH_20190926T124734_20190926T124804_029192_0350B9_FA6B',
        # # Antarctica - Heard, McDonald Island (-53.1057, 73.5431) 
        'S1A_IW_SLC__1SSH_20230127T142750_20230127T142817_046970_05A22F_17F7',
        'S1A_IW_SLC__1SSH_20230620T142747_20230620T142817_049070_05E69E_0BC7',
        # Antarctica - Brunt Ice Shelf (-75.216667, -25.683333) 
        'S1B_IW_SLC__1SSH_20210223T233056_20210223T233124_025740_031194_E7BE',
        'S1B_IW_SLC__1SSH_20210228T035005_20210228T035033_025801_03138F_8CB2',
        # Antarctica - Victoria Land, East Ross Sea (-72.5, 168) 
        'S1A_IW_SLC__1SSH_20230120T160153_20230120T160220_046869_059EC6_F364',
        'S1A_IW_SLC__1SSH_20230116T100627_20230116T100655_046807_059CB3_FCC7',
]
softwares = ['pyrosar','rtc-opera','hyp3-gamma', 'S1_NRB']
dems = ['glo_30','REMA_32']

# get crededentials for AWS
with open('aws_credentials.txt') as f:
    ACCESS_ID, ACCESS_KEY = f.readlines()
    ACCESS_ID = ACCESS_ID.strip()
    ACCESS_KEY = ACCESS_KEY.strip()

# setup s3
s3 = boto3.client('s3', 
                        region_name='ap-southeast-2',
                        aws_access_key_id=ACCESS_ID,
                        aws_secret_access_key= ACCESS_KEY)

# make data directory to store local files
os.makedirs('data', exist_ok=True)

# Functions

In [ ]:
def get_scene_poly_from_asf(scene):
    # get the scene geom in 4326
    asf.constants.CMR_TIMEOUT = 45
    asf_result = asf.granule_search([scene], asf.ASFSearchOptions(processingLevel='SLC'))[0]
    points = (asf_result.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
                    ['Geometry']['GPolygons'][0]['Boundary']['Points'])
    points = [(p['Longitude'],p['Latitude']) for p in points]
    scene_poly = Polygon(points)
    return scene_poly

## Show example scene files for software

**Products**
- RTC products are stored in the s3 bucket specified above
- The general pattern for the products is : ```software/dem/crs/scene/files```

**Projections**
- pyrosar and rtc-opera are projected in 3031 polar steregraphic coordinates 
- hype-gamma and S1_NRB products are provided in UTM zone projections (output crs cannot be specified)
- S1_NRB products are split accross crs zones, with auxillary products produced using the sentinel-2 MRGS tiling grid
    - A singe scene ID is therefore split across multiple crs folders
    - aux products are written to the final crs folder for each product

**Data**
- Naming convention for rtc backscatter differs for each product

**Metadata**
- Metadata products are stored differently between products

In [ ]:
file_list = []
for software in softwares:
    for dem in dems:
        params = {
            "Bucket": f'{s3_bucket}',
            "Prefix": f'{software}/{dem}'
        }
        objects = s3.list_objects_v2(**params)
        if 'Contents' in objects.keys():
            data = objects['Contents']
            file_list.extend([x for x in objects['Contents']])

# save all of the files in a dataframe for east of searching
df_s3 = pd.DataFrame.from_records(file_list)
df_s3[['software','dem','crs','scene','file']] = df_s3['Key'].str.split('/', n=4, expand=True)
df_s3 = df_s3.sort_values(['software','dem','crs','scene','file'])
# filter for scenes we're interested in
df_s3 = df_s3[df_s3['scene'].isin(scenes)]
df_s3.head(2)

In [ ]:
print('software for each scene')
df_s3.groupby('scene')['software'].unique()

In [ ]:
print('Examples of projections for a given RTC product')
df_s3[((df_s3['scene']==scenes[4]) & (df_s3['dem']=='glo_30'))].groupby(['software','scene'])['crs'].unique().reset_index()

# Compare total timing

In [ ]:
# limit to single dem and proj
df_timing_files = df_s3[(
    (df_s3['file'].str.contains('_timing.json'))
    #& (df_s3['dem']==dem)
    #& (df_s3['crs']==proj
    )]
df_timing_files.head(2)

In [ ]:
timing_data = []
for i in range(0,len(df_timing_files)):
    timing_file = df_timing_files.iloc[i].Key
    #print(timing_file)
    try:
        s3.download_file(s3_bucket, timing_file, 'tmp.json')
        with open('tmp.json') as json_file:
            data = json.load(json_file)
            data['software'] = df_timing_files.iloc[i].software
            data['scene'] = df_timing_files.iloc[i].scene
        timing_data.append(data)
        #print(f'downloaded: {timing_file}')
    except:
        ...
        #print(f'no timing file: {timing_file}')

os.remove('tmp.json')
df_timing = pd.DataFrame.from_records(timing_data, index=['software','scene'])

# gamma values are in list for some reason
for col in list(df_timing):
    df_timing[col] = df_timing[col].apply(lambda x : x[0] if isinstance(x,list) else x)

# min timing filter to remove false information
min_time = 1000
df_timing = df_timing[df_timing['Total']>min_time]

# plot mean time by software
sw_count = df_timing.groupby('software').size()
ax = (df_timing.groupby('software').mean()
 .drop(columns=['Total'])
 .plot.bar(stacked=True))
ax.set_xlabel('Software')
ax.set_ylabel('Time (seconds)')
ax.set_title(f'Software Processing Mean Times (DEM upsampling=2)')

In [ ]:
sw_count.plot(kind='bar', title='valid product timing file count by software')

# Relative Comparison of RTC Scenes

## Helper functions

In [ ]:
def filter_df(df_s3, scene, software, dem, crs_list, file_endswith, file_contains):
    df_filter = df_s3[(
            (df_s3['scene'] == scene)
            & (df_s3['software'] == software) 
            & (df_s3['dem'] == dem)
            & (df_s3['crs'].isin(crs_list))
            & df_s3['file'].str.endswith(file_endswith)
            & df_s3['file'].str.contains(file_contains)
            )
        ]
    return df_filter

def download_files(df, s3_bucket, local_folder, exist_ok=True):
    """downloads files in the passed df from s3 bucket 

    Args:
        df (pd.DataFrame): dataframe with s3 info
        s3_bucket (str): name of the s3 bucket
        local_folder (str): local folder to save file
        exist_ok (bool, optional): Skip if file exists. Defaults to True.
    """

    for i in range(0,len(df)):
        key = df.iloc[i].Key
        filename = df.iloc[i].file
        local_file = os.path.join(local_folder, filename)
        if exist_ok:
            if not os.path.exists(local_file):
                print(f'downloading {filename}')
                s3.download_file(s3_bucket, key, local_file)
            else:
                print(f'skipping download, already exists: {filename}')
        else:
            print(f'downloading {filename}')
            s3.download_file(s3_bucket, key, local_file)
    
    # add metadata to df
    df['local_file'] = df['file'].apply(lambda x : os.path.join(local_folder, filename))
    # drop any hdr files after downloading - these provide metadata to the .img file
    df = df[~(df['file'].str.endswith('hdr'))]
    return df

View the options of products to compare a single scene (software, crs, dem)

In [ ]:
scene = 'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B'
# view the available software, crs and dem for the scene
df_s3[df_s3['scene']==scene].groupby(['software','crs','dem'])['file'].count()

- For this analysis, the dependant variable is the software. 
- The DEM is fixed as the cop glo_30 dem. 
- CRS differs between the software. rtc-opera and pyrosar are already projected in the target crs 3031. 
- S1_NRB has multiple crs. We select the same crs as output by the hype-gamma software (default UTM zone).

In [ ]:
dem_1 = dem_2 = 'glo_30'
target_crs = 3031

output_dir = 'data/compare-rtc'
download_dir = 'data/tifs'

# make data directory to store local files
os.makedirs(output_dir, exist_ok=True)
os.makedirs(download_dir, exist_ok=True)

# stop pandas warning when setting values
pd.options.mode.chained_assignment = None

# Create pairwise tuples for all unique combinations
pairwise_software = list(combinations(softwares, 2))
for i,scene in enumerate(scenes):
        # iterate through each scene
    for software_1, software_2 in reversed(pairwise_software):
        # iterate through the comparison pairs
        # set the crs we are interested in, some messy logic used for time being
        if software_1 in ['pyrosar','rtc-opera']:
            crs_1 = target_crs
        if software_2 in ['pyrosar','rtc-opera']:
            crs_2 = target_crs
        if str(software_1) in ['hyp3-gamma','S1_NRB']:
            crs_1 = df_s3[((df_s3['software']=='hyp3-gamma') & (df_s3['scene']==scene))]['crs'].values[0]
        if str(software_2) in ['hyp3-gamma','S1_NRB']:
            crs_2 = df_s3[((df_s3['software']=='hyp3-gamma') & (df_s3['scene']==scene))]['crs'].values[0]
        
        # the dems for s1_nrb are saved in the last crs folder, the count of files in this folder
        # will be highest
        if (software_1 == 'S1_NRB' or software_2 == 'S1_NRB'):
            dem_crs =  df_s3[
                ((df_s3['software']=='S1_NRB') & (df_s3['scene']==scene))
                ].groupby('crs').count().sort_values('file',ascending=False).index[0]
        else:
            dem_crs = ''
            
        label_1 = f'{software_1}-{dem_1}-{crs_1}'
        label_2 = f'{software_2}-{dem_2}-{crs_2}'

        print('Relative Comparison')
        print(f'SCENE 1 - {label_1}')
        print(f'SCENE 2 - {label_2}')
        print(f'All pixel based comparisons will be undertaken at target crs : {target_crs}')

        rtc_df_1 = filter_df(df_s3, scene, software_1, dem_1, [str(crs_1), str(dem_crs)], ('tif','img','hdr'), 'rtc|HH')
        rtc_df_2 = filter_df(df_s3, scene, software_2, dem_2, [str(crs_2), str(dem_crs)], ('tif','img','hdr'), 'rtc|HH')
        dem_df_1 = filter_df(df_s3, scene, software_1, dem_1, [str(crs_1), str(dem_crs)], ('tif','img','hdr'), 'rtc|HH')
        dem_df_2 = filter_df(df_s3, scene, software_2, dem_2, [str(crs_2), str(dem_crs)], ('_dem.tif','-em.tif'), '')

        # download the tifs
        print('\nDownloding rtc tifs for comparison')
        rtc_df_1 = download_files(rtc_df_1, s3_bucket, download_dir)
        rtc_df_2 = download_files(rtc_df_2, s3_bucket, download_dir)

        save_folder = os.path.join(output_dir,scene,f'{label_1}_vs_{label_2}')
        os.makedirs(save_folder, exist_ok=True)

        # patch the missing crs for pyrosar
        for f in [rtc_df_1, rtc_df_2]:
            if f.iloc[0].software == 'pyrosar':
                assign_crs(f.iloc[0].local_file, f.iloc[0].crs,)
        
        # plot the two tiffs side by side with native projections, shapes etc
        rtc_file_1 = rtc_df_1.iloc[0].local_file
        rtc_file_2 = rtc_df_2.iloc[0].local_file
        
        plot_tifs(
            rtc_file_1, 
            rtc_file_2, 
            titles= [label_1, label_2],
            suptitle=f'{scene}',
            convert_dB = True,
            cmap = 'binary_r',
            scale=[-40,10],
            save_path=os.path.join(save_folder,'original_crs_compare.jpg'))
        
        plot_histograms(
            tifs=[rtc_file_1,rtc_file_2],
            titles= [label_1, label_2],
            colours=['red','blue'],
            suptitle=f'{scene}',
            convert_dB=True,
            save_path=os.path.join(save_folder,'original_crs_distributions.jpg'))
        
        # get the scene polygon 
        scene_poly = get_scene_poly_from_asf(scene)

        # reproject and match the tifs
        rtc_file_1_reproj = os.path.join(save_folder,f'{label_1}_reproj_db.tif')
        rtc_file_2_reproj = os.path.join(save_folder,f'{label_2}_reproj_db.tif')
        
        reproject_match_tifs(
            rtc_file_1, 
            rtc_file_2, 
            target_crs,
            scene_poly=scene_poly,
            save_path_1=rtc_file_1_reproj,
            save_path_2=rtc_file_2_reproj,
            convert_dB=True,
            set_nodata=np.nan)

        #calculate the difference
        print('Calculating raster difference')
        rtc_file_diff = os.path.join(save_folder,f'difference_db.tif')
        diff = rioxarray.open_rasterio(rtc_file_1_reproj) - rioxarray.open_rasterio(rtc_file_2_reproj)
        diff.rio.to_raster(rtc_file_diff)

        plot_histograms(
            tifs=[rtc_file_diff],
            titles= f'difference',
            colours=['black'],
            suptitle=f'difference ([{label_1}] - [{label_2}])',
            convert_dB=False,
            save_path=os.path.join(save_folder,'difference_distribution.jpg'))
        
        plot_array(
            rtc_file_diff, 
            cmap='bwr',
            vmin=-1,
            vmax=1,
            title=f'abs difference ([{label_1}] - [{label_2}])',
            ylabel='Decibel Difference (db)',
            save_path=os.path.join(save_folder,f'difference_plot.jpg'))

        # plot the difference maps
        scales = [[-40,10],[-40,10],[-1,1]]
        titles = [f'{label_1}-reproj',
                f'{label_2}-reproj',
                f'abs difference ([{label_1}] - [{label_2}])']
        
        plot_difference_maps(
            rtc_file_1_reproj, 
            rtc_file_2_reproj,
            titles=titles,
            scales=scales,
            save_path=os.path.join(save_folder,f'difference_summary_plot.jpg')
            )

        # make a gif that shows the geometric distance
        bounds = [5000,5500,5000,5500]
        make_difference_gif(
            rtc_file_1_reproj, 
            rtc_file_2_reproj, 
            vmin=-10, 
            vmax=40, 
            bounds=bounds, 
            title='Image geometric offset',
            save_path=os.path.join(save_folder,f'offset.gif'))
        
        #coregister with arosics
        import arosics
        # img_trg is the one that is shifted
        # move image 2 to align with image 1

        rtc_file_2_reproj_matched = os.path.join(save_folder,f'{label_2}_reproj_matched_db.tif')
        coreg = arosics.CoReg.COREG(
            im_ref=rtc_file_1_reproj, 
            im_tgt=rtc_file_1_reproj, 
            path_out=rtc_file_2_reproj_matched, 
            fmt_out='GTIFF',
            align_grids=True)
        
        res = coreg.correct_shifts()

        # movement statistics 
        offest_stats = {
            'x_shift_px': coreg.x_shift_px,
            'y_shift_px': coreg.y_shift_px,
            'x_shift_map': coreg.x_shift_map,
            'y_shift_map': coreg.y_shift_map,
            'vec_length_map': coreg.vec_length_map,
            'vec_angle_deg': coreg.vec_angle_deg,
        }
        
        # this can slightly change the shapes, ensure tifs are same shape tio snip the same spot
        print('re-aligning shapes')
        tif_2_matched =  (
            rioxarray.open_rasterio(rtc_file_2_reproj_matched)
            .rio.reproject_match(rioxarray.open_rasterio(rtc_file_1_reproj))
        ).rio.to_raster(rtc_file_2_reproj_matched)

        make_difference_gif(
            rtc_file_1_reproj, 
            rtc_file_2_reproj_matched, 
            vmin=-10, 
            vmax=40, 
            bounds=bounds, 
            title='Image geometric offset after coregistration',
            save_path=os.path.join(save_folder,f'offset_after_coreg.gif'))
    
        break
    
    break

In [ ]:
# make the settings for the comparison
# compare 1 scene agains a single dependant variable (e.g. dem, software, proj)
# scene 1 variables
software_1 = 'S1_NRB'
dem_1 = 'glo_30'
crs_1 = 32733
label_1 = f'{software_1}-{dem_1}-{crs_1}'
# scene 2 variables
software_2 = 'pyrosar'
dem_2 = 'glo_30'
crs_2 = 3031
label_2 = f'{software_2}-{dem_2}-{crs_2}'
# set the target crs
target_crs = 3031

print('Relative Comparison')
print(f'SCENE 1 - {label_1}')
print(f'SCENE 2 - {label_2}')
print(f'All pixel based comparisons will be undertaken at target crs : {target_crs}')

In [ ]:
df_comparison = df_s3[(
    (df_s3['scene'] == scene)
    & (
        (((df_s3['software'] == software_1) & (df_s3['dem'] == dem_1) & (df_s3['crs'].isin([crs_1,str(crs_1)])))
        |
        ((df_s3['software'] == software_2) & (df_s3['dem'] == dem_2) & (df_s3['crs'].isin([crs_2,str(crs_2)])))
    ))
)
]
scene_tifs = df_comparison[(
      (df_comparison.file.str.contains('rtc|HH')) &
      (df_comparison.file.str.endswith(('tif','img','hdr')))
      )]
scene_dems = df_comparison[(df_comparison.file.str.endswith(('_dem.tif','-em.tif')))]
print(f'{len(scene_tifs)} scene tifs found meeting conditions')
print(f'{len(scene_dems)} scene dems found meeting conditions')

# determine which scenes need to be reprojected
scene_tifs['reproject'] = scene_tifs['crs'].apply(lambda x : False if str(x) == str(target_crs) else True)
scene_tifs[['software','dem','crs','Size','reproject','file']]

## Download the tifs

In [ ]:
# download tifs and store locally
download = True
exist_ok = True
if download:
    for i in range(0,len(scene_tifs)):
        key = scene_tifs.iloc[i].Key
        filename = scene_tifs.iloc[i].file
        local_file = f'data/tifs/{filename}'
        if exist_ok:
            if not os.path.exists(local_file):
                print(f'downloading {filename}')
                s3.download_file(s3_bucket, key, local_file)
            else:
                print(f'skipping download, already exists: {filename}')
        else:
            print(f'downloading {filename}')
            s3.download_file(s3_bucket, key, local_file)

scene_tifs['local_file'] = scene_tifs['file'].apply(lambda x : f'data/tifs/{x}')
# drop any hdr files after downloading
scene_tifs = scene_tifs[~(scene_tifs['file'].str.endswith('hdr'))]

## Comparison of outputs in native crs

In [ ]:
# plot the two tiffs side by side with native projections, shapes etc
plot_tifs(
    scene_tifs.iloc[0].local_file, 
    scene_tifs.iloc[1].local_file, 
    titles= [label_1, label_2],
    suptitle=f'{scene}',
    convert_dB = True,
    cmap = 'binary_r',
    scale=[-40,10])

## Comparison of rasters in target crs
- Project rasters to the same resolutions and shapes to enable a direct difference
- Note differences may be due to geometric differences and not intensity
- Be sure to inspect the pixel shift below

In [ ]:
scene_poly = get_scene_poly_from_asf(scene)

In [ ]:
# local files
print(scene_tifs.iloc[0].local_file)
print(scene_tifs.iloc[1].local_file)
# assign crs to missing pyrosar if it does not already have
# this is an error with pyrosar not correctly assigning the crs

for i in range(len(scene_tifs)):
    if scene_tifs.iloc[i].software == 'pyrosar':
        print(f'assigning crs to {scene_tifs.iloc[i].local_file}')
        assign_crs(scene_tifs.iloc[i].local_file, scene_tifs.iloc[i].crs,)

tif_1_matched, tif_2_matched = reproject_match_tifs(
    scene_tifs.iloc[0].local_file, 
    scene_tifs.iloc[1].local_file, 
    scene_poly, 
    target_crs)

print('converting from linear to db')
tif_1_db = 10*np.log10(tif_1_matched)
tif_2_db = 10*np.log10(tif_2_matched)

del tif_1_matched, tif_2_matched

# # calculate the difference between the two images
diff = tif_1_db - tif_2_db
tif_1_db.rio.to_raster(f'data/tifs/{scene}_{label_1}_clipped.tif')
tif_2_db.rio.to_raster(f'data/tifs/{scene}_{label_2}_clipped.tif')
diff.rio.to_raster(f'data/tifs/{scene}_diff_clipped.tif')

# add location to df
scene_tifs['diff_db_path'] = f'data/tifs/{scene}_diff_clipped.tif'
scene_tifs['local_matched_db_path'] =  0
scene_tifs['local_matched_db_path'].iloc[0] = f'data/tifs/{scene}_{label_1}_clipped.tif'
scene_tifs['local_matched_db_path'].iloc[1]= f'data/tifs/{scene}_{label_2}_clipped.tif'
scene_tifs

In [ ]:
# resample 
# upscale_factor = 0.1
upscale_factor = False
if upscale_factor:
    new_width = int(tif_1_db.rio.width * upscale_factor)
    new_height = int(tif_1_db.rio.height * upscale_factor)

    tif_1_db = tif_1_db.rio.reproject(
        tif_1_db.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
    )

    tif_2_db = tif_2_db.rio.reproject(
        tif_2_db.rio.crs,
        shape=(new_height, new_width),
        resampling=Resampling.bilinear,
    )

    diff = tif_1_db - tif_2_db
    print(diff.shape)

In [ ]:
scales = [[-40,10],[-40,10],[-1,1]]
titles = [f'{label_1}-reproj',
          f'{label_2}-reproj',
          f'abs difference ([{label_1}] - [{label_2}])']
plot_difference_maps(
      tif_1_db, 
      tif_2_db,
      titles=titles,
      scales=scales,
      save_path=f'data/compare-rtc/{scene}_{label_1}_vs_{label_2}.png')

In [ ]:
a = rioxarray.open_rasterio('/Users/alexbradley/Documents/AlexsComputer/GA/Projects/compare-RTC/data/tifs/hyp3-gamma-glo_30-32733-S1B_IW_20190315T195015_SHP_RTC20_G_gpuned_E650_dem.tif')

## Visualise geometric shift between results

In [ ]:
x1,x2,y1,y2 = 8600,9000,6600,7000 # full res
#x1,x2,y1,y2 = 3400,4000,9400,10000 # full res
#x1,x2,y1,y2 = 6500,6900,4600,5000 # full res
#x1,x2,y1,y2 = 2000,2500,8600,9100 # full res
#x1,x2,y1,y2 = 7000,7500,3800,4300# full res
if upscale_factor:
    x1,x2,y1,y2 = [int(n*upscale_factor) for n in [x1,x2,y1,y2]] # adjust for scaling
tif_1_snip = tif_1_db[0][y1:y2,x1:x2]
tif_2_snip = tif_2_db[0][y1:y2,x1:x2]
animation = make_gif(
    [tif_2_snip, tif_1_snip], 
    vmin=-40, 
    vmax=10, 
    title=f'{dependant_vals[0]}_vs_{dependant_vals[1]}')
animation.save(f'data/compare-rtc/{scene}_{label_1}_vs_{label_2}.gif')
HTML(animation.to_html5_video())

## Coregtister the Images with Arosics

In [ ]:
import arosics
# img_trg is the one that is shifted
# move image 2 to align with image 1
scene_tifs['local_coreg_db_path'] = 0
scene_tifs['local_coreg_db_path'].iloc[0]= f'data/tifs/{scene}_{label_1}_clipped.tif'
scene_tifs['local_coreg_db_path'].iloc[1]= f'data/tifs/{scene}_{label_2}_coreg.tif'

coreg = arosics.CoReg.COREG(
    im_ref=f'{scene_tifs.iloc[0].local_matched_db_path}', 
    im_tgt=f'{scene_tifs.iloc[1].local_matched_db_path}', 
    path_out=f'{scene_tifs["local_coreg_db_path"].iloc[1]}', 
    fmt_out='GTIFF',
    align_grids=True)
res = coreg.correct_shifts()

# movement statistics 
offest_stats = {
    'x_shift_px': coreg.x_shift_px,
    'y_shift_px': coreg.y_shift_px,
    'x_shift_map': coreg.x_shift_map,
    'y_shift_map': coreg.y_shift_map,
    'vec_length_map': coreg.vec_length_map,
    'vec_angle_deg': coreg.vec_angle_deg,
}

# this can slightly change the shapes, ensure tifs are same shape
print('re-aligning shapes')
tif_2_matched =  (
    rioxarray.open_rasterio(scene_tifs['local_coreg_db_path'].iloc[1])
    .rio.reproject_match(rioxarray.open_rasterio(scene_tifs['local_coreg_db_path'].iloc[0]))
)
tif_2_matched.rio.to_raster(scene_tifs['local_coreg_db_path'].iloc[1])

In [ ]:
print(f'Movement from {label_2} to {label_1}')
for k in offest_stats.keys():
    print(f'{k}: {offest_stats[k]}')

In [ ]:
tif_1_db = rioxarray.open_rasterio(f'{scene_tifs["local_coreg_db_path"].iloc[0]}')
tif_2_db = rioxarray.open_rasterio(f'{scene_tifs["local_coreg_db_path"].iloc[1]}')

tif_1_snip = tif_1_db[0][y1:y2,x1:x2]
tif_2_snip = tif_2_db[0][y1:y2,x1:x2]
animation = make_gif(
    [tif_2_snip, 
    tif_1_snip], 
    vmin=-40, 
    vmax=10, 
    title=f'{label_1}_vs_{label_2}_aligned')
animation.save(f'data/compare-rtc/{scene}_{label_1}_vs_{label_2}_aligned.gif')
HTML(animation.to_html5_video())

In [ ]:
tif_1_db.shape, tif_2_db.shape

In [ ]:
scales = [[-40,10],[-40,10],[-1,1]]
titles = [f'{label_1}-coreg',f'{label_2}_coreg',
          f'abs difference ({label_1}-coreg - {label_2}_coreg)']
plot_difference_maps(
      tif_1_db, 
      tif_2_db,
      titles=titles,
      scales=scales,
      save_path=f'data/compare-rtc/{scene}_{label_1}_vs_{label_2}_coreg.png')

# Relative comparison of DEMS

In [ ]:
# download tifs and store locally
local_dems = []
download = True
exist_ok=True
if download:
      for i in range(0,len(scene_dems)):
            key = scene_dems.iloc[i].Key
            filename = scene_dems.iloc[i].file
            print(filename)
            sw = scene_dems.iloc[i].software
            label = f'{scene_dems.software.iloc[i]}-{scene_dems.dem.iloc[i]}-{scene_dems.crs.iloc[i]}'
            local_file =  f'data/tifs/{label}-{filename}'
            if exist_ok:
                  if not os.path.exists(local_file):
                        print(f'downloading {filename}')
                        s3.download_file(s3_bucket, key, local_file)
                  else:
                        print(f'skipping download, already exists: {filename}')
            else:
                  print(f'downloading {filename}')
                  s3.download_file(s3_bucket, key, local_file)
scene_dems['label'] = scene_dems[['software', 'dem', 'crs']].agg('-'.join, axis=1)
scene_dems['local_file'] = f'data/tifs/' + scene_dems['label'] + '-' + scene_dems['file']
scene_dems

In [ ]:
# if multiple dems just pic first...
scene_dems = scene_dems[scene_dems['file'].isin(scene_dems.groupby(['software'])['file'].first().values)].sort_values('software')

In [ ]:
# clear mem
tif_1_db = tif_2_db = coreg = None

In [ ]:
# get the scene geom in 4326
asf.constants.CMR_TIMEOUT = 45
asf_result = asf.granule_search([scene], asf.ASFSearchOptions(processingLevel='SLC'))[0]
points = (asf_result.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
                ['Geometry']['GPolygons'][0]['Boundary']['Points'])
points = [(p['Longitude'],p['Latitude']) for p in points]
scene_poly = Polygon(points)
str(scene_poly)

In [ ]:
# load in the dems
print(f'{scene_dems["local_file"].iloc[0]}')
print(f'{scene_dems["local_file"].iloc[1]}')

tif_1_dem_matched, tif_2_dem_matched = reproject_match_tifs(
    scene_dems.iloc[0].local_file, 
    scene_dems.iloc[1].local_file, 
    scene_poly, 
    target_crs)

In [ ]:
# convert to arrays and make nodata consistent 
tif_1_dem = np.array(tif_1_dem_matched).astype(np.float32)
tif_2_dem = np.array(tif_2_dem_matched).astype(np.float32)
tif_2_dem[(tif_2_dem==tif_2_dem_matched.rio.nodata)] = np.nan
tif_1_dem[(tif_1_dem==tif_1_dem_matched.rio.nodata)] = np.nan
tif_2_dem[(tif_2_dem==0)] = np.nan
tif_1_dem[(tif_1_dem==0)] = np.nan

In [ ]:
# try and adjust for pixel size (30m/20)
x1,x2,y1,y2 = (np.array((x1,x2,y1,y2))*(1/3)).astype(int)
print(x1,x2,y1,y2)
tif_1_dem_snip = tif_1_dem[0][y1:y2,x1:x2]
tif_2_dem_snip = tif_2_dem[0][y1:y2,x1:x2]
# replace pyrosar nodata of -9999 with nans
animation = make_gif([tif_2_dem_snip, tif_1_dem_snip], vmin=0, vmax=None)
HTML(animation.to_html5_video())

In [ ]:
scales = [[0,700],[0,700],[-30,30]]
titles = [f'{scene_dems["label"].iloc[0]}',
          f'{scene_dems["label"].iloc[1]}',
          f'abs difference ({scene_dems["label"].iloc[0]} - {scene_dems["label"].iloc[1]})']
ylabels = ['elevation (m)', 'elevation (m)', 'elevation difference (m)']
plot_difference_maps(
      tif_1_dem, 
      tif_2_dem,
      titles=titles,
      scales=scales,
      ylabels=ylabels,
      exclude_nodata=True
      )

# Scratch

In [ ]:
comparison_data = {

    'scene' : ...,
    'file_1' : ...,
    'file_2' : ...,
    'software_1' : ...,
    'software_2' : ...,
    'dem_1' : ...,
    'dem_2' : ...,
    'crs_1' : ...,
    'crs_2' : ...,
    'label_1' : ...,
    'label_2' : ...,
    'target_crs' : ...,

    'rtc_db_1_min' : ...,
    'rtc_db_1_max' : ...,
    'rtc_db_1_mean' : ...,
    'rtc_db_1_median' : ...,
    'rtc_db_1_std' : ...,
    'rtc_db_1_5p' : ...,
    'rtc_db_1_95p' : ...,

    'rtc_db_2_min' : ...,
    'rtc_db_2_max' : ...,
    'rtc_db_2_mean' : ...,
    'rtc_db_2_median' : ...,
    'rtc_db_2_std' : ...,
    'rtc_db_2_5p' : ...,
    'rtc_db_2_95p' : ...,

    'rtc_db_diff_min' : ...,
    'rtc_db_diff_max' : ...,
    'rtc_db_diff_mean' : ...,
    'rtc_db_diff_median' : ...,
    'rtc_db_diff_std' : ...,
    'rtc_db_diff_5p' : ...,
    'rtc_db_diff_95p' : ...,

    'coreg_x_shift_px': ...,
    'coreg_y_shift_px': ...,
    'coreg_x_shift_map': ...,
    'coreg_y_shift_map': ...,
    'coreg_vec_length_map': ...,
    'coreg_vec_angle_deg': ...,

    'dem_1_file' : ...,
    'dem_2_file' : ...,
    'dem_1_crs' : ...,
    'dem_2_crs' : ...,

    'dem_1_min' : ...,
    'dem_1_min' : ...,
    'dem_1_max' : ...,
    'dem_1_mean' : ...,
    'dem_1_median' : ...,
    'dem_1_std' : ...,
    'dem_1_5p' : ...,
    'dem_1_95p' : ...,

    'dem_2_min' : ...,
    'dem_2_min' : ...,
    'dem_2_max' : ...,
    'dem_2_mean' : ...,
    'dem_2_median' : ...,
    'dem_2_std' : ...,
    'dem_2_5p' : ...,
    'dem_2_95p' : ...,

    'dem_diff_min' : ...,
    'dem_diff_min' : ...,
    'dem_diff_max' : ...,
    'dem_diff_mean' : ...,
    'dem_diff_median' : ...,
    'dem_diff_std' : ...,
    'dem_diff_5p' : ...,
    'dem_diff_95p' : ...,
}

In [ ]:
def generate_extra_points_along_boundary(bbox, delta=0.1):
    """
    Generate points along the boundary of a bounding box.

    Parameters:
    - bbox: Tuple of four coordinates (x_min, y_min, x_max, y_max).
    - delta: distance between points along the bounding box sides 

    Returns:
    - List of points [(x1, y1), (x2, y2), ...] along the boundary.
    """
    x_min, y_min, x_max, y_max = bbox
    # Generate points along the top side
    top_side = [(x, y_max) for x in list(np.arange(x_min, x_max, delta)) + [x_max]]    
    # Generate points along the right side
    right_side = [(x_max, y) for y in list(np.arange(y_max - delta, y_min-delta, -delta)) + [y_min-delta]]
    # Generate points along the bottom side
    bottom_side = [(x, y_min) for x in list(np.arange(x_max - delta, x_min-delta, -delta)) + [x_min-delta]]
    list(np.arange(y_min + delta, y_max, delta)) + [y_max]
    # Generate points along the left side
    left_side = [(x_min, y) for y in list(np.arange(y_min + delta, y_max, delta)) + [y_max]]
    # Combine all sides' points
    all_points = top_side + right_side + bottom_side + left_side
    return all_points

# Example usage:
bounding_box = (150, -75, 160, -70)
points_along_boundary = generate_extra_points_along_boundary(bounding_box)
print(points_along_boundary)
Polygon(points_along_boundary)
